In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

from skyfield.api import Loader, EarthSatellite
from skyfield.timelib import Time
from skyfield.api import utc

import plotly as py
import plotly.graph_objs as go
import cv2

load = Loader('../../data/SkyData')
data = load('de421.bsp')
ts   = load.timescale()

In [3]:
import sys
sys.path.append('../..')
from pkg.orbital_congestion import socrates

socrates_files_path = '../../data/socrates/'
tle_file_path = '../../data/socrates_tca_gp_history_tle.pkl.gz'

soc_df, tle_df = socrates.get_all_socrates_and_tle_data(socrates_files_path, tle_file_path)

In [4]:
def get_earth_texture(size):
    # Texture
    img = cv2.imread('../3dearth/Textures/800px-Mercator_Projection.png')
    display(img.shape)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to greyscale
    img = cv2.resize(img, (size,size), interpolation = cv2.INTER_AREA)  #resize image to match graph linspace
    img = np.fliplr(img) # flip the image
    axis = 1
    #img = np.roll(img, int(img.shape[axis]/2), axis=axis) # roll the image
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE) # rotate the image
    surfcolor = img

    # Coloring
    pl_grey =[[0.0, 'rgb(0, 0, 0)'],
     [0.05, 'rgb(13, 13, 13)'],
     [0.1, 'rgb(29, 29, 29)'],
     [0.15, 'rgb(45, 45, 45)'],
     [0.2, 'rgb(64, 64, 64)'],
     [0.25, 'rgb(82, 82, 82)'],
     [0.3, 'rgb(94, 94, 94)'],
     [0.35, 'rgb(108, 108, 108)'],
     [0.4, 'rgb(122, 122, 122)'],
     [0.45, 'rgb(136, 136, 136)'],
     [0.5, 'rgb(150, 150, 150)'],
     [0.55, 'rgb(165, 165, 165)'],
     [0.6, 'rgb(181, 181, 181)'],
     [0.65, 'rgb(194, 194, 194)'],
     [0.7, 'rgb(206, 206, 206)'],
     [0.75, 'rgb(217, 217, 217)'],
     [0.8, 'rgb(226, 226, 226)'],
     [0.85, 'rgb(235, 235, 235)'],
     [0.9, 'rgb(243, 243, 243)'],
     [0.95, 'rgb(249, 249, 249)'],
     [1.0, 'rgb(255, 255, 255)']]
    return surfcolor, pl_grey

def draw_earth():
    size = 400  # resolution of earth - higher looks nicer but its slower
    earth_radius = 6378.

    # Coefficients
    coefs = (1, 1, 1) 
    rx, ry, rz = [earth_radius/np.sqrt(coef) for coef in coefs]

    theta = np.linspace(0,2*np.pi,size)
    phi = np.linspace(0,np.pi,size)
    x = rx * np.outer(np.cos(theta),np.sin(phi))
    y = ry * np.outer(np.sin(theta),np.sin(phi))
    z = rz * np.outer(np.ones(size),np.cos(phi))  # note this is 2d now
    
    # Earth texture
    surfcolor, pl_grey = get_earth_texture(size)

    data = go.Surface(
        x=x,
        y=y,
        z=z,
        surfacecolor=surfcolor,
        colorscale=pl_grey,
        showscale=False
    )

    layout = go.Layout(
        title='Earth',
        autosize=False,
        width=800,
        height=700,
        margin=go.layout.Margin(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

def plot_each_near_collisions(fig, tle_df):
    df = tle_df[tle_df['sat1_tle'].notnull()][['sat1_tle', 'tca_time']]
    nc_xyz = []
    for idx, row in tqdm(df.iterrows()):
        sat = EarthSatellite(*row['sat1_tle'].split(','))
        tca = row['tca_time']
        tca = tca.replace(tzinfo=utc)
        time = ts.utc(tca)
        nc_xyz.append(sat.at(time).position.km)

    x,y,z = zip(*nc_xyz)
    data = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name='Near-Collisions',
        mode='markers',
        marker=dict(
            color='red',
            size=3
        )
    )
    fig.add_trace(data)
    
    return fig

In [ ]:
print('Drawing earth...')
fig = draw_earth()
print(f'Drawing {len(tle_df)} near collisions...')
fig = plot_each_near_collisions(fig, tle_df)
print('Done')
fig